# <center>Google Brain - Ventilator Pressure Prediction<center>
## <center>Starter: LSTM Neural Network (TPU)<center>
---
This is my first notebook with a LSTM Network and also the first time I ran a model on TPU. As the intent of this notebook is to provide a ‘framework’ for future ones, in this version (could be updated), I’m not using all the engineered features available on the top public notebooks, only those I managed to understand until the moment I created this.


Suggestion of EDA notebook in this dataset:
* [Ventilator Pressure: EDA and simple submission](https://www.kaggle.com/carlmcbrideellis/ventilator-pressure-eda-and-simple-submission) by [Carl McBride Ellis](https://www.kaggle.com/carlmcbrideellis)
    
Notebook that inspired my work:
* [Finetune of Tensorflow Bidirectional LSTM](https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm) by [zhangxin](https://www.kaggle.com/tenffe)

## Importing Packages and Datasets

In [ ]:
import pandas as pd       
import matplotlib as mat
import matplotlib.pyplot as plt    
import numpy as np
import seaborn as sns
%matplotlib inline

import random
import os

from numpy.random import seed
seed(42)

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model


from tensorflow.random import set_seed
set_seed(42)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv', index_col = 'id')
X_test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv', index_col = 'id')
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

X_train = df_train.copy().drop('pressure', axis = 1)
Y_train = df_train['pressure'].copy()

In [ ]:
df_train

In [ ]:
X_test

## Feature Engineering + Reshaping the Dataset

In [ ]:
#Inspired by https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/273974
X_train['u_in_cumsum'] = (X_train['u_in']).groupby(X_train['breath_id']).cumsum()
X_test['u_in_cumsum'] = (X_test['u_in']).groupby(X_test['breath_id']).cumsum()

#Lag features. 
#Number of lag features inspired by https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm
X_train['u_in_lag1'] = X_train.groupby('breath_id')['u_in'].shift(1).fillna(0)
X_test['u_in_lag1'] = X_test.groupby('breath_id')['u_in'].shift(1).fillna(0)
X_train['u_in_lag2'] = X_train.groupby('breath_id')['u_in'].shift(2).fillna(0)
X_test['u_in_lag2'] = X_test.groupby('breath_id')['u_in'].shift(2).fillna(0)
X_train['u_in_lag3'] = X_train.groupby('breath_id')['u_in'].shift(3).fillna(0)
X_test['u_in_lag3'] = X_test.groupby('breath_id')['u_in'].shift(3).fillna(0)
X_train['u_in_lag4'] = X_train.groupby('breath_id')['u_in'].shift(4).fillna(0)
X_test['u_in_lag4'] = X_test.groupby('breath_id')['u_in'].shift(4).fillna(0)

X_train['u_in_lag-1'] = X_train.groupby('breath_id')['u_in'].shift(-1).fillna(0)
X_test['u_in_lag-1'] = X_test.groupby('breath_id')['u_in'].shift(-1).fillna(0)
X_train['u_in_lag-2'] = X_train.groupby('breath_id')['u_in'].shift(-2).fillna(0)
X_test['u_in_lag-2'] = X_test.groupby('breath_id')['u_in'].shift(-2).fillna(0)
X_train['u_in_lag-3'] = X_train.groupby('breath_id')['u_in'].shift(-3).fillna(0)
X_test['u_in_lag-3'] = X_test.groupby('breath_id')['u_in'].shift(-3).fillna(0)
X_train['u_in_lag-4'] = X_train.groupby('breath_id')['u_in'].shift(-4).fillna(0)
X_test['u_in_lag-4'] = X_test.groupby('breath_id')['u_in'].shift(-4).fillna(0)

#Window features
#Choice of features inspired vy https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm
X_train['breath_id__u_in__max'] = X_train.groupby(['breath_id'])['u_in'].transform('max')
X_test['breath_id__u_in__max'] = X_test.groupby(['breath_id'])['u_in'].transform('max')
X_train['breath_id__u_in__mean'] = X_train.groupby(['breath_id'])['u_in'].transform('mean')
X_test['breath_id__u_in__mean'] = X_test.groupby(['breath_id'])['u_in'].transform('mean')

X_train['breath_id__u_out__max'] = X_train.groupby(['breath_id'])['u_out'].transform('max')
X_test['breath_id__u_out__max'] = X_test.groupby(['breath_id'])['u_out'].transform('max')
X_train['breath_id__u_out__mean'] = X_train.groupby(['breath_id'])['u_out'].transform('mean')
X_test['breath_id__u_out__mean'] = X_test.groupby(['breath_id'])['u_out'].transform('mean')

X_train = X_train.drop(['breath_id'], axis = 1)
X_test = X_test.drop(['breath_id'], axis = 1)


X_train

In [ ]:
scaler = RobustScaler()

X_train[X_train.columns] = scaler.fit_transform(X_train[X_train.columns])
X_test[X_test.columns] = scaler.transform(X_test[X_test.columns])

In [ ]:
X_train

In [ ]:
#Original set lenght
train_len = len(X_train)
test_len = len(X_test)

In [ ]:
#Reshaping Data
#80 - timesteps
Y_train = Y_train.values.reshape(-1,80)
X_train = X_train.values.reshape(-1,80,18)
X_test = X_test.values.reshape(-1,80,18)

print(X_train.shape, Y_train.shape, X_test.shape)

## Defining and Running the Neural Network

In [ ]:
#Defining Callbacks

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    min_delta=0.0000001,
    restore_best_weights=True,
)

plateau = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor = 0.5,                                     
    patience = 3,                                   
    min_delt = 0.0000001,                                
    cooldown = 0,                               
    verbose = 1
)

In [ ]:
#Model
def new_model():

    inputs = layers.Input(shape = (80,18))
    
    x = layers.Bidirectional(keras.layers.LSTM(1024, return_sequences=True))(inputs)
    x = layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True))(x)
    x = layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True))(x)
    x = layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(x)
    x = layers.Dense(128, activation='selu')(x)
    
    #Final Layer (Output)
    output = layers.Dense(1, activation = 'linear')(x)
    
    model = keras.Model(inputs=[inputs], outputs=output)
    
    return model

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():
        
    kfold = KFold(n_splits = 5, shuffle=True, random_state = 42)

    pred = np.zeros((test_len,1))
    #train_oof = np.zeros((train_len,1))
    train_oof = np.zeros((len(X_train),80,1))

    for fold, idx in enumerate(kfold.split(X=X_train, y=Y_train)):
        
        print(20*'--','Fold: ', fold+1, 20*'--')
        
        train_idx, val_idx = idx[0], idx[1]
        xtrain = X_train[train_idx]
        ytrain = Y_train[train_idx]
        xval = X_train[val_idx]
        yval = Y_train[val_idx]

        model = new_model()
        model.compile(loss='mae', optimizer = keras.optimizers.Adam(learning_rate=0.002),
                      metrics=[keras.metrics.MeanAbsoluteError()])
        
        checkpoint_filepath = f"folds{fold}.hdf5"
        
        check = keras.callbacks.ModelCheckpoint(
            checkpoint_filepath,
            monitor='val_loss',
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='auto',
            save_freq='epoch',
            options=None
        )

        model.fit(xtrain, ytrain,
        batch_size = 1024, epochs = 250,
        validation_data=(xval, yval),
        callbacks=[early_stopping, plateau, check]);

        #create predictions
        pred += model.predict(X_test).reshape(-1,1)/kfold.n_splits
        #print(pred)

        val_pred = model.predict(xval)

        # getting out-of-fold predictions on training set
        train_oof[val_idx] = val_pred

## Generating Outputs

In [ ]:
train_oof = train_oof.reshape(-1,1)
Y_train_reshaped = Y_train.reshape(-1,1)

In [ ]:
print("OOF - MAE: {0:0.6f}".format(mean_absolute_error(Y_train_reshaped,train_oof)))

In [ ]:
train_oof = pd.DataFrame(train_oof, columns = ['pressure'])
train_oof.to_csv('train_oof.csv', index=False)

train_oof

In [ ]:
submission['pressure'] = pred
submission.to_csv("submission.csv", index=False)
submission